In [1]:
import numpy as np
import pandas as pd

In [4]:
data_imdb = r"https://drive.google.com/file/d/1QtSgzb9eF0cqXBd0cFLpDlBG9LfBh5UK/view?usp=sharing"
data_imdb = r"https://drive.google.com/uc?id=" + data_imdb.split('/')[-2]
data_imdb = pd.read_csv(data_imdb,delimiter='\t',header=None)
data_imdb.columns = ["Review_text","Review class"]

In [5]:
data_amazon = r"https://drive.google.com/file/d/1czEh8Ga2Uo2omdeMZECbQsjtKFqomq1O/view?usp=sharing"
data_amazon = r"https://drive.google.com/uc?id=" + data_amazon.split('/')[-2]
data_amazon = pd.read_csv(data_amazon,delimiter='\t',header=None)
data_amazon.columns = ["Review_text","Review class"]

In [6]:
data_yelp = r"https://drive.google.com/file/d/1_ev3N05EjCy3P3vm5D_nSSG9GlTf2uDJ/view?usp=sharing"
data_yelp = r"https://drive.google.com/uc?id=" + data_yelp.split('/')[-2]
data_yelp = pd.read_csv(data_yelp,delimiter='\t',header=None)
data_yelp.columns = ["Review_text","Review class"]

In [ ]:
data = pd.concat([data_imdb,data_amazon,data_yelp])
data

In [ ]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
def clean_text(df):
    all_reviews = list()
    lines = df["Review_text"].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
#         words = [w for w in words if not w in stop_words]
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

all_reviews = clean_text(data)
all_reviews[0:20]

In [ ]:
stopwords.words("english")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(min_df=3)   
X = CV.fit_transform(all_reviews).toarray()
y = data['Review class'].values
print(np.shape(X))
print(np.shape(y))

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier(criterion="entropy", random_state=41)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, precision_score
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))

0.6618181818181819
0.7038216560509554
0.6314285714285715
